## <span style="color:#3333cc"> **SQL Demonstration Summary**  </span>

### <span style="color:#5c5cd6"> Recreated simple database from bootcamp project on Render </span>

### <span style="color:#8585e0"> SQL queries written to incorporate: </span>
* group by with aggregate functions
* Union all
* time range (simple - year)
* cast
* Round results
* column alias
* multi-table join (default - inner)
* lowercase
* order by
* limit to 20 results
        

### <span style="color:#8585e0"> Previous experience with: </span>
time ranges using datetime, trim, substring, like, wildcards, operators 

#### <span style="color:#8585e0"> DB reference:</span>

https://github.com/ChristyGruen/meteorite-ml-project


In [1]:
# import dependencies
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float
import psycopg2
from psycopg2 import Error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from config import aws_pw, pg_pw, render_pw

In [2]:
# RENDER POSTGRES DB CONNECTION
# https://naysan.ca/2020/05/31/postgresql-to-pandas/
param_dic = {
    "user"      : "chrisministrator",
    "password"  : render_pw,
    "host"      : "dpg-cgkt61orddleudolgrng-a.oregon-postgres.render.com",  
    "port"      : "5432",
    "database"  : "meteorites"
}
def connect(params_dic):
    """ Connect to the PostgreSQL/RENDER database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL/RENDER database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [3]:
# function to query database and put data into df
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [4]:
# connect to database
conn = connect(param_dic)


Connecting to the PostgreSQL/RENDER database...
Connection successful


In [5]:
# create column names for summary query
summary_query_cols = ['category','value', 'min_mass_gr', 'avg_mass_gr','max_mass_gr']

### SUMMARY SQL QUERY contains:

    group by with aggregate functions
    Union all
    time range (simple - year)
    cast
    Round results
    column alias
    2 table join (default - inner)
        

select 'meteorite_class' as category, mt.meteorite_class as value, min(mm.mass_grams) as min_mass_gr, round(cast(avg(mm.mass_grams)as numeric),0) as avg_mass_gr, max(mm.mass_grams) as max_mass_gr<br>
from meteorite_main mm, meteorite_type mt<br>
where<br>
mm.recclass = mt.recclass<br>
and <br>
mm.year > 1900<br>
group by mt.meteorite_class

union all

select 'state' as category, s.state as value, min(mm.mass_grams) as min_mass_gr, round(cast(avg(mm.mass_grams)as numeric),0) as avg_mass_gr, max(mm.mass_grams) as max_mass_gr<br>
from meteorite_main mm, state s<br>
where<br>
mm.state_abbrev = s.state_abbrev<br>
and <br>
mm.year > 1900<br>
group by s.state

In [6]:
# SUMMARY SQL QUERY from SummaryQuery.sql

summary_query = postgresql_to_dataframe(conn,"select 'meteorite_class' as category, mt.meteorite_class as value, min(mm.mass_grams) as min_mass_gr, round(cast(avg(mm.mass_grams)as numeric),0) as avg_mass_gr, max(mm.mass_grams) as max_mass_gr from meteorite_main mm, meteorite_type mt where mm.recclass = mt.recclass and mm.year > 1900 group by mt.meteorite_class union all select 'state' as category, s.state as value, min(mm.mass_grams) as min_mass_gr, round(cast(avg(mm.mass_grams)as numeric),0) as avg_mass_gr, max(mm.mass_grams) as max_mass_gr from meteorite_main mm, state s where mm.state_abbrev = s.state_abbrev and mm.year > 1900 group by s.state",summary_query_cols) 
summary_query.head(20)

,category,value,min_mass_gr,avg_mass_gr,max_mass_gr
0,meteorite_class,Mesosiderite,11.70,12652,38690.0
1,meteorite_class,Achrondrite,1.77,45314,1100000.0
2,meteorite_class,Chrondrite,0.50,9621,700000.0
3,meteorite_class,Pallasite,40.70,8841,37600.0
4,meteorite_class,Iron,12.00,160110,15500000.0
5,state,Oklahoma,78.40,37845,270000.0
6,state,Colorado,6.10,11808,309000.0
7,state,North Carolina,668.00,19802,72700.0
8,state,Mississippi,280.00,313,345.0
9,state,Florida,180.00,10956,41800.0


### General Query contains
    lowercase
    3 table join (default - inner)
    order by
    limit to 20 results

select mm.mass_grams, lower(mm.fall), mm.year, mm.reclat, mm.reclong, mm.elevation, s.state_abbrev, s.FIPS, s.area_sqkm, mt.meteorite_class<br>
from meteorite_main mm, meteorite_type mt, state s<br>
where<br>
mm.state_abbrev = s.state_abbrev<br>
and<br>
mm.recclass = mt.recclass<br>
and <br>
mm.year > 1900<br>
order by s.state_abbrev, mm.year,mt.meteorite_class, mm.mass_grams

In [7]:
general_query_cols = ['mass_grams', 'fall', 'year', 'reclat', 'reclong', 'elevation','state_abbrev', 'FIPS', 'area_sqkm', 'meteorite_class']

In [8]:
# GENERAL QUERY from GeneralQuery.sql

general_query = postgresql_to_dataframe(conn,"select mm.mass_grams, lower(mm.fall), mm.year, mm.reclat, mm.reclong, mm.elevation, s.state_abbrev, s.FIPS, s.area_sqkm, mt.meteorite_class from meteorite_main mm, meteorite_type mt, state s where mm.state_abbrev = s.state_abbrev and mm.recclass = mt.recclass and mm.year > 1900 order by s.state_abbrev, mm.year,mt.meteorite_class, mm.mass_grams limit 20", general_query_cols) 
general_query

,mass_grams,fall,year,reclat,reclong,elevation,state_abbrev,FIPS,area_sqkm,meteorite_class
0,320.0,found,1921,55.18333,-162.55000,-6.0,AK,2,1723337,Pallasite
1,43000.0,found,1942,64.88333,-163.16667,134.0,AK,2,1723337,Iron
2,9830.0,found,1998,65.38333,-146.26667,707.0,AK,2,1723337,Chrondrite
3,140600.0,found,1906,32.40000,-87.00000,35.0,AL,1,135767,Chrondrite
4,877.0,fell,1907,34.58333,-87.50000,188.0,AL,1,135767,Chrondrite
5,265.0,fell,1933,34.75000,-87.00000,204.0,AL,1,135767,Chrondrite
6,94000.0,found,1935,32.00000,-86.00000,144.0,AL,1,135767,Iron
7,5560.0,fell,1954,33.18836,-86.29450,178.0,AL,1,135767,Chrondrite
8,140000.0,found,1957,34.68333,-85.65000,466.0,AL,1,135767,Iron
9,34500.0,found,1969,33.96667,-87.91667,144.0,AL,1,135767,Iron


In [9]:
# close connection
conn.close()